# Unsolvable Loop Analysis

All the functionality described in this notebook is from our paper called [(Un)Solvable Loop Analysis](https://arxiv.org/abs/2306.01597).
Check it out for more details on the theory behind what is illustrated in this notebook.

Polar usually requires that the programs it analyzes satisfy some requirements (see [README.md](../README.md)).
One of the requirements is that there are no non-linear cyclic dependencies. This sounds more complicated than it actually is.
Let's look at an example of a loop that violates this requirement and is called *unsolvable*:

In [2]:
!cat loops/markov-triples-random.prob

a, b, c = 1, 1, 2
while true:
    d = Bernoulli(1/2)
    if d == 1:
        a, b, c = a, 3*a*b - c, b
    else:
        a, b, c = b, 3*b*c - a, c
    end
end


The loop has program variables `a,b,c` and `d`. The variable `d` just draws from `Bernoulli` distribution.
However, the other three variables are all mutually dependent: `a` depends on `b`, `b` depends on `a`, `b` and `c`, `c` depends on `b`.
Hence, there are multiple cycles in the dependency graph. A cycle means we can start a variable follow other variables it depends on and eventually get back to where we started.
Moreover, some dependencies in these cycles are non-linear because of the assignments `3*a*b - c` and `3*b*c - a`.
This means the loop contains non-linear cyclic dependencies and hence the loop is *unsolvable*. In this example the variable `d` is unproblematic, we call it *effective*, whereas the variables `a,b,c` are problematic and called `defective`. Polar cannot analyze unsolvable loops the same way as it analyzes solvable loops. If we would just blindly run Polar on this loop for computing an expected value, it would not terminate.
We can pass the additional parameter `solvability_check` to Polar and instead of running forever Polar will tell us that is loop us indeed unsolvable.

In [5]:
!python ../polar.py loops/markov-triples-random.prob --goals "E(c)" --solvability_check


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

Traceback (most recent call last):
  File "/home/marcel/Repos/polar/documentation/../polar.py", line 35, in <module>
    main()
  File "/home/marcel/Repos/polar/documentation/../polar.py", line 27, in main
    raise e
  File "/home/marcel/Repos/polar/documentation/../polar.py", line 25, in main
    action(benchmark)
  File "/home/marcel/Repos/polar/cli/actions/goals_action.py", line 54, in __call__
    self.handle_all_goals()
  File "/home/marcel/Repos/polar/cli/act

## Synthesize Unsolvable Invariants

However, not everything is lost. Although we cannot compute the moments of individual variables, Polar can search for polynomials in the defective variables that have a closed-form formula.
For this, we pass the parameter `synth_unsolv_inv` (for "synthesizing unsolvable invariant") and a maximum degree for the polynomial we want to look for (through the parameter `inv_deg`).

In [10]:
!python ../polar.py loops/markov-triples-random.prob --synth_unsolv_inv --inv_deg 2


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

Searching for invariants for special case k = 1..
No invariant found with degree 2 and k=1

Searching for invariants, general case..
E(-_u27*(a - c)*(a + c)) = 3*2**(-n)*_u27
E(-(a - c)*(_u24 + _u27*a + _u27*c)) = 2**(-n)*(_u24 + 3*_u27)
Elapsed time: 0.9417839050292969 s


Polar has found two classes of quadratic polynomials in `a`, `b`, and `c` together with their closed-form formulas (of the expected value), although each variable individually does not admit a closed-form formula.
The symbolic constants (beginning with an underscore) in the formulas represent any real number. So, for every instantiation of these symbols with a real number we have a polynomial in the defective variables.
whose expected value admits a closed-form formula.

## Synthesize Solvable from Unsolvable Loop

Polar can also synthesize a solvable loop from an unsolvable loop. To do so, Polar first computes "well-behaved" polynomials for the unsolvable loop as illustrated above. Then it deletes all defective variables in the unsolvable loop and introduces a new variable (begins with `_s`) that models the found "well-behaved" polynomial in defective variables. For every "well-behaved" polynomials of defective variables Polar synthesize one loop. Because we previously got two classes of polynomials Polar synthesizes two loops (for the maximum degree 2).
The synthesized loops are always non-probabilistic. For effective variables, the variables in the synthesized loops model the expected values of the original variables.

In [11]:
!python ../polar.py loops/markov-triples-random.prob --synth_solv_loop --inv_deg 2


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

Synthesized solvable loop: 
_t22 = d0
_s20 = 3*_u17
d = _t22
while true:
    _t22 = 1/2
    _s20 = (1/2)*_s20
    d = _t22
end

Invariant used: 
(-a**2*_u17 + c**2*_u17, 3*2**(-n)*_u17)


Synthesized solvable loop: 
_t23 = d0
_s21 = _u14 + 3*_u17
d = _t23
while true:
    _t23 = 1/2
    _s21 = (1/2)*_s21
    d = _t23
end

Invariant used: 
(-a*_u14 - a**2*_u17 + c*_u14 + c**2*_u17, 2**(-n)*(_u14 + 3*_u17))


Elapsed time: 0.8217787742614746 s


## In Python

We can perform the same computation in Python code instead of the CLI.
We first load the program, decide among which variables Polar should look for a "well-behaved" polynomial and up to which maximum degree.

In [6]:
from inputparser import Parser
from program import normalize_program
from unsolvable_analysis import UnsolvInvSynthesizer

# Load program and convert to normal form
program = Parser().parse_file("loops/markov-triples-random.prob")
program = normalize_program(program)

# We want to look for "well-behaved" polynomials among all program variables that are defective
# That is a,b and c in our example
defective_vars = [
    v for v in program.original_variables if v in program.defective_variables
]
# Search for polynomials up to a maximum degree of two.
inv_deg = 2
solutions = UnsolvInvSynthesizer.synth_inv(defective_vars, inv_deg, program)
print(solutions)

[(a**2*_u167 - c**2*_u167, -3*2**(-n)*_u167), (a*_u164 + a**2*_u167 - c*_u164 - c**2*_u167, 2**(-n)*(-_u164 - 3*_u167))]


We get the same two classes of polynomials together with their closed-form formulas as we found through the command line interface.
We can synthesize solvable loops from the unsolvable loop similarly:

In [9]:
from inputparser import Parser
from program import normalize_program
from unsolvable_analysis import SolvLoopSynthesizer

program = Parser().parse_file("loops/markov-triples-random.prob")
program = normalize_program(program)
defective_vars = [
    v for v in program.original_variables if v in program.defective_variables
]
inv_deg = 2
invariants, solvable_programs = SolvLoopSynthesizer.synth_loop(
    defective_vars, inv_deg, program
)

for invariant, solvable_program in zip(invariants, solvable_programs):
    print("Synthesized solvable loop: ")
    print(solvable_program)
    print()
    print("Invariant used: ")
    print(invariant)
    print()
    print()

Synthesized solvable loop: 
_t229 = d0
_s227 = -3*_u224
d = _t229
while true:
    _t229 = 1/2
    _s227 = (1/2)*_s227
    d = _t229
end

Invariant used: 
(a**2*_u224 - c**2*_u224, -3*2**(-n)*_u224)


Synthesized solvable loop: 
_t230 = d0
_s228 = -_u221 - 3*_u224
d = _t230
while true:
    _t230 = 1/2
    _s228 = (1/2)*_s228
    d = _t230
end

Invariant used: 
(a*_u221 + a**2*_u224 - c*_u221 - c**2*_u224, 2**(-n)*(-_u221 - 3*_u224))


